### Edgar Moises Hernandez-Gonzalez
#### 21/11/19
#### Clasificacion de RWE-DWT de MI-EEG con CNN-1D + LSTM
##### Validacion cruzada de 5
##### Optimizar hiperparametros con busqueda de cuadricula

In [0]:
from google.colab import drive

In [0]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Conv1D, MaxPool1D, Flatten, LSTM, Dense, Dropout
from tensorflow.keras.wrappers import scikit_learn
from sklearn.model_selection import GridSearchCV

In [0]:
x_train = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RWE_DWT/MI-EEG-RWE-A01T.csv", header=None)
x_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RWE_DWT/MI-EEG-RWE-A01E.csv", header=None)
y_train = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RWE_DWT/EtiquetasA01T.csv", header=None)
y_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RWE_DWT/EtiquetasA01E.csv", header=None)

In [0]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(288, 66)
(288, 66)
(288, 1)
(288, 1)


In [0]:
x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))

print(x.shape)
print(y.shape)

(576, 66)
(576, 1)


In [0]:
n_samples = len(y)

print("n_samples:", n_samples)

n_samples: 576


In [0]:
n_classes = len(np.unique(y_test)) #esto se podria calcular asi n_clases = len(np.unique(y)), la y puede ser train o test

print("n_classes:", n_classes)

n_classes: 4


In [0]:
#escalar los datos para que tengan media = 0 y desviacion estandar = 1
scaler = StandardScaler().fit(x)
x = scaler.transform(x)

In [0]:
#codificar etiquetas con valores de 0 a nClases-1
le = LabelEncoder().fit(y)
y = le.transform(y)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
#convertir de 2D a 4D
x = x.reshape((n_samples, 1, 3, 22))

print(x.shape)

(576, 1, 3, 22)


In [0]:
def model(n_filtros, n_unidades):
    model = Sequential()
    model.add(TimeDistributed(Conv1D(n_filtros, 3, activation='relu'), input_shape=(None,3,22)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(n_unidades))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [0]:
clf = scikit_learn.KerasClassifier(build_fn = model, epochs=20, batch_size=18, verbose=0)

In [0]:
n_filtros = [16, 32]
n_unidades = [16, 32, 64, 128]
hiperparametros = dict(n_filtros=n_filtros, n_unidades=n_unidades)

In [0]:
clf = GridSearchCV(clf, hiperparametros, cv=5)

In [0]:
clf.fit(x, y)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


KeyboardInterrupt: ignored

In [0]:
print(clf.best_params_)
print(clf.best_score_)

{'n_filtros': 32, 'n_unidades': 64}
0.5017361053679552


In [0]:
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
        % (mean, std * 2, params))

0.465 (+/-0.045) for {'n_filtros': 16, 'n_unidades': 16}
0.474 (+/-0.048) for {'n_filtros': 16, 'n_unidades': 32}
0.443 (+/-0.073) for {'n_filtros': 16, 'n_unidades': 64}
0.481 (+/-0.102) for {'n_filtros': 16, 'n_unidades': 128}
0.486 (+/-0.060) for {'n_filtros': 32, 'n_unidades': 16}
0.497 (+/-0.061) for {'n_filtros': 32, 'n_unidades': 32}
0.502 (+/-0.082) for {'n_filtros': 32, 'n_unidades': 64}
0.479 (+/-0.084) for {'n_filtros': 32, 'n_unidades': 128}
